# Transformaciones

In [ ]:
import json
import pandas as pd
import re
from textblob import TextBlob
from ast import literal_eval
from datetime import datetime

### 1- Dataset User_reviews.json
    Contiene datos de las recomendaciones y comentarios de los jugadores

In [ ]:
##Cargamos los datos desde la carpeta Datasets
##Obserbamos que el Dataset tiene columnas con datos anidados en forma de diccionarios por lo que
##deberemos tratarlo linea por linea

# Creamos esta funcion para corregir y cargar cada linea del archivo 'australian_user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# lista para almacenar los datos desanidados
data_list = []

# extraemos números de la columna 'funny'
numero_regex = re.compile(r'\d+')

# lee el archivo 'australian_user_reviews.json' original, corrige y procesa cada línea
with open('./Datasets/australian_user_reviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            user_id = entrada['user_id']
            user_url = entrada['user_url']
            # iteramos sobre cada reseña
            for reseña in entrada['reviews']:
                # estraemos el numero de la columna 'funny'
                funny_valor = re.search(numero_regex, reseña.get('funny', ''))
                if funny_valor:
                    funny = int(funny_valor.group())
                else:
                    funny = None
                
                # elimina 'Posted' de la columna 'posted'
                posted = reseña.get('posted', '').replace('Posted ', '', 1) 

                reseña_dict = {
                    'user_id': user_id,
                    'user_url': user_url,
                    'funny': funny,
                    'posted': posted,
                    'item_id': int(reseña.get('item_id', '')),  # convierte a entero
                    'helpful': reseña.get('helpful', ''),
                    'recommend': reseña.get('recommend', ''),
                    'review': reseña.get('review', '')  # mantiene el texto original de la review
                }
                data_list.append(reseña_dict)
        except json.JSONDecodeError as e:
            None

# crea el DataFrame
df = pd.DataFrame(data_list)



In [ ]:
df.columns

In [ ]:
# Reemplaza los valores NaN por None en el DataFrame
df = df.where(df.notna(), None)
df


In [ ]:
#La columna ['posted'] presenta fechas escritas y algunas sin el año, 
# deduzco que el año corresponde al año en curzo en el momento de la extraccion de los datos
#por lo que realizaremos la extraccion, transformacion y asignacion del año correcto 

# Definimos una función para extraer y convertir la fecha
def extract_date(posted_str):
    if not isinstance(posted_str, str):
        return None
    
    # Elimina el punto y espacios en blanco al principio y al final
    date_str = posted_str.replace('.', '').strip()
    try:
        # Intenta convertir la cadena de fecha en un objeto de fecha utilizando datetime.strptime
        date_obj = datetime.strptime(date_str, '%B %d, %Y').date()
    except ValueError:
        try:
            # Intenta nuevamente con el formato '%B %d' si la conversión falla
            date_obj = datetime.strptime(date_str, '%B %d').date()
            # Si solo tiene mes y día y no se proporcionó el año, asumimos el año 2016
            date_obj = date_obj.replace(year=2016)
        except ValueError:
            # Si no se puede convertir a ninguna de las dos formas, devuelve None
            date_obj = None

    return date_obj

# Aplica la función extract_date a la columna 'posted'
df['posted'] = df['posted'].apply(extract_date)

# Mostrar las primeras filas para verificar
print(df)
